# SkipGram with Negative Sampling (SGNS)


## History
The SkipGram Model with negative sampling (also know as Word2Vec) was introduced by Mikolov et al. [1,2] in 2013.
SkipGramModel implements the skipgram with negative sampling architecture from word2vec (an input word should predict an output word in the context). However, the core component dates back to the 50ies. It is based on the "distributional hyposthesis"[3], which states that words in a similar context tend to have a similar meaning.
Also the idea of learning word embeddings had been around before [4], but high computational costs hindered their widespread. Mikolov presented a shallow model, which could still learn good representations, but at low computational costs. This and the open release of the implementation can be seen as the success factors of Word2Vec. The widespread most likely got an additional boost by the Python implementation of Radim Řehůřek [gensim](https://radimrehurek.com/gensim/).

[1] Mikolov, Tomas, et al. "Efficient estimation of word representations in vector space." ICLR. 2013.  
[2] Mikolov, Tomas, et al. "Distributed representations of words and phrases and their compositionality." Advances in neural information processing systems. 2013.  
[3] Harris, Zellig S. "Distributional structure." Word 10.2-3 (1954): 146-162.  
[4] Bengio, Yoshua, et al. "A neural probabilistic language model." Journal of machine learning research 3.Feb (2003): 1137-1155.

## Model
The goal is to find an embedding for the words $\phi : w \in W \rightarrow \mathbb{R}^{|W| \times d}$, 
where $d << |W|$.
We want to embed the words into a lower-dimensional, real-valued space, while still retaining information from original space.
Given a word $u$, we aim to estimate the likelihood to see another word $v$ co-occur:
$p(w_v|w_u)$

We can move towards this goal, by maximizing the likelihood of words in the context:  
$\sum_{u \in W} log P(v \in C^+|\phi(u))$  
That is, we aim to maximize the log-probability of observing a word $v$ that resides in the context (denoted as $C^+$) of $u$, conditioned on its feature representation $\phi(u)$.

## Definition of Context - Sliding Window
We move a sliding window with pre-determined size across a sentence. The middle word is our word of interest and the words to the left and right, which are still within the window are considered as context. We generate context pairs from this sliding window approach. To put more weight on the immediate neighbors, Mikolov proposed to randomly reduce the window size to sample far distant words less often.

In [1]:
import numpy as np

def sliding_window(sentence,window_size=2,downsample=True):
    pairs = []
    words = [w for w in sentence]
    for pos, word in enumerate(words):
        reduction = 0
        if downsample:
            reduction = np.random.randint(window_size)
        start = max(0, pos - window_size + reduction)
        for pos2, word2 in enumerate(words[start:(pos + window_size + 1 - reduction)], start):
            if pos2 != pos:
                pairs.append((word, word2))        
    return pairs

In [2]:
sliding_window("the cat sat on the mat".split(),downsample=False,window_size=2)

[('the', 'cat'),
 ('the', 'sat'),
 ('cat', 'the'),
 ('cat', 'sat'),
 ('cat', 'on'),
 ('sat', 'the'),
 ('sat', 'cat'),
 ('sat', 'on'),
 ('sat', 'the'),
 ('on', 'cat'),
 ('on', 'sat'),
 ('on', 'the'),
 ('on', 'mat'),
 ('the', 'sat'),
 ('the', 'on'),
 ('the', 'mat'),
 ('mat', 'on'),
 ('mat', 'the')]

## Training - Shallow Neural Network
We can now use the source-context word pairs to train a neural network with our previously defined optimization objective. We model the conditional log-likelihood of every source-context pair as a softmax unit, parametrized by a dot product of the words' feature representations:

$P(v^+|\phi(u)) = \frac{exp(\langle \phi^\prime(v^+), \phi(u) \rangle)}{\sum_{v \in W} exp(\langle \phi^\prime(v), \phi(u) \rangle)}$

where $\langle \cdot \rangle$ is the dot product and $\phi^\prime$ is a similar mapping as $\phi$, often referred to as projection (opposed to embedding). Technically, this is implemented by a shallow neural network, with a linear hidden layer, where the embeddings are the weights between the input layer and the hidden layer and the projections are the weights between the hidden layer and the output layer.

However, the normalization in the denominator is costly to compute and therefore, we replace $P(v^+|\phi(u))$ by negative sampling, as proposed by Mikolov et al. 

$ log \ p(w_v|w_u) \approx log \ \sigma(\langle \phi^\prime(v),\phi(u)\rangle) + \sum_{k=1}^{K} \mathbb{E}_{w_k \sim P_n(w)}[log \ \sigma (-\langle \phi^\prime(w_k),\phi(u)\rangle]$  
with  
$\langle \cdotp \rangle$ the dot product  
$\sigma(x) = \frac{1}{1+exp(-x)}$ the logistic sigmoid function  
$K$ the number of negative samples, drawn from the sampling distribution  
$P_n(w), \forall w \in W$ where  
$W$ is the vocabularity and the probability of drawing a word should correspond to it's frequency. 

This substitution of the softmax by negative sampling is possible, as we are not interested in actual probabilities, but only aim to find a good embedding. In fact, Mikolov et al. showed empirically that negative sampling yields even better embeddings than the softmax.



### The SkipGram Network

In [3]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


class SkipGramNetwork(nn.Module):

    def __init__(self, emb_size_u, emb_size_v, emb_dimension):
        super(SkipGramNetwork, self).__init__()
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(emb_size_u, emb_dimension,sparse=True)
        self.v_embeddings = nn.Embedding(emb_size_v, emb_dimension,sparse=True)
        self.init_emb()

    def init_emb(self):
        initrange = 0.5 / self.emb_dimension
        self.u_embeddings.weight.data.uniform_(-initrange, initrange)
        self.v_embeddings.weight.data.uniform_(-0, 0)
        
    def forward(self, pos_u, pos_v, neg_v):
        emb_u = self.u_embeddings(pos_u)
        emb_v = self.v_embeddings(pos_v)
        emb_n = self.v_embeddings(neg_v)
        pos_score = F.logsigmoid(torch.mm(emb_v,torch.t(emb_u)))
        neg_score = F.logsigmoid(torch.mm(emb_n,torch.t(emb_u)).neg())
        score = torch.sum(pos_score) + torch.sum(neg_score)
        return -1 * score

### Pre-Processing Utility
In order to prepare our data to be fed into the neural network, we need to:
- transform words to integer ids
- create context pairs by sliding window
- downsample high frequent words (see below)
- setup the noise distribution for sampling the negative words (Mikolov empirically showed, that the unigram distribution raised to the power of 0.75 is a good choice)

**Downsampling high frequent words**
High frequent words are words like "a", "the", ... (stopwords). They usually provide less information than rare words, but due to their frequency see way more updates. To counter this imbalance, each word in the training set is discarded with probability
$P(w_i)= 1-\sqrt{\frac{t}{f(w_i)}}$  
where $f(w_i)$ is the frequency of the word and $t$ a threshold, typically around $10^{-5}$. This formula aggressively downsamples high frequent words.

In [7]:
from collections import defaultdict
from gensim.models.doc2vec import TaggedDocument
import random

class WVDataset():
    def __init__(self, sentences, power=0.75,window_size=5,neg_samples=5,sample=1e-3,min_count=5):
        self.sentences = sentences
        self.window_size = window_size
        self.neg_samples = neg_samples
        self.vocab = None
        self.index2source = dict()
        self.source2index = dict()
        self.index2context = dict()
        self.context2index = dict()
        self.build_vocab(sentences,min_count)
        self.ctx_weights = self.make_freq_table(power)
        self.neg_sample_buffer = []
        # downsample frequent words
        self.downsample_probs = np.zeros_like(self.ctx_weights)
        if sample > 0:
            self.downsample_probs = 1 - np.sqrt(sample/self.ctx_weights).clip(0,1)
       
        
    def build_vocab(self,sentences,min_count):
        # word freqency
        raw_vocab = defaultdict(int)
        for sent in sentences:
            for word in sent:
                raw_vocab[word] += 1 
        # only keep words that occur at least min_count times
        self.vocab = {k:v for k,v in raw_vocab.items() if v >= min_count}
        del raw_vocab
        # map each word to an integer
        for word in self.vocab:
            self.source2index[word] = len(self.source2index)
        # reverse mapping (integer to word)
        self.index2source = dict(zip(self.source2index.values(), self.source2index.keys()))
        # in- and output layer are the same in word2vec
        self.context2index = self.source2index
        self.index2context = self.source2index
            
        
    def make_freq_table(self, power):
        # the unigram distribution raised to 0.75 empirically performed best as negative sampling distribution
        pow_frequency = np.array([self.vocab[self.index2source[i]] for i in range(len(self.vocab))])**power
        return pow_frequency / pow_frequency.sum()
        

    def sliding_window(self, words):
        for pos, word in enumerate(words):
            # sliding window (randomly reduced to give more weight to closeby words)
            reduction = np.random.randint(self.window_size)
            start = max(0, pos - self.window_size + reduction)
            for pos2, word2 in enumerate(words[start:(pos + self.window_size + 1 - reduction)], start):
                if pos2 != pos:
                    yield self.source2index[word],self.source2index[word2]
    
    def negative_sampling(self,context):
        neg = []
        while len(neg) < self.neg_samples:
            try:
                negative = self.neg_sample_buffer.pop()
                # avoid that context word occurs in negative samples
                if negative != context:
                    neg.append(negative)
            except IndexError:
                # instead of sampling negative words for each context pair individually, we sample a large set of negative samples at once (faster)
                self.neg_sample_buffer = list(np.random.choice(list(self.index2source.keys()),size=1000,replace=True,p=self.ctx_weights))
        return neg
        
    
    def get_pairs(self):
        for sent in self.sentences:
            # randomly reject frequent words according to downsampling probability
            words = [w for w in sent if w in self.vocab and self.downsample_probs[self.source2index[w]] < random.random()]

            for source,context in self.sliding_window(words):
                yield source,context,self.negative_sampling(context)        

#### Quick Test

In [6]:
from gensim.utils import tokenize

sents = [
    "the cat sat on the mat",
    "the cat sat on the table",
    "the fox sat on the mat",
    "a car was broken",
    "the fox sat on the table"
]


test_sents = [list(tokenize(s)) for i in range(10) for s in sents] # tokenize and repeat to increase corpus size

In [206]:
test_sents

[['the', 'cat', 'sat', 'on', 'the', 'mat'],
 ['the', 'cat', 'sat', 'on', 'the', 'table'],
 ['the', 'fox', 'sat', 'on', 'the', 'mat'],
 ['a', 'car', 'was', 'broken'],
 ['the', 'fox', 'sat', 'on', 'the', 'table'],
 ['the', 'cat', 'sat', 'on', 'the', 'mat'],
 ['the', 'cat', 'sat', 'on', 'the', 'table'],
 ['the', 'fox', 'sat', 'on', 'the', 'mat'],
 ['a', 'car', 'was', 'broken'],
 ['the', 'fox', 'sat', 'on', 'the', 'table'],
 ['the', 'cat', 'sat', 'on', 'the', 'mat'],
 ['the', 'cat', 'sat', 'on', 'the', 'table'],
 ['the', 'fox', 'sat', 'on', 'the', 'mat'],
 ['a', 'car', 'was', 'broken'],
 ['the', 'fox', 'sat', 'on', 'the', 'table'],
 ['the', 'cat', 'sat', 'on', 'the', 'mat'],
 ['the', 'cat', 'sat', 'on', 'the', 'table'],
 ['the', 'fox', 'sat', 'on', 'the', 'mat'],
 ['a', 'car', 'was', 'broken'],
 ['the', 'fox', 'sat', 'on', 'the', 'table'],
 ['the', 'cat', 'sat', 'on', 'the', 'mat'],
 ['the', 'cat', 'sat', 'on', 'the', 'table'],
 ['the', 'fox', 'sat', 'on', 'the', 'mat'],
 ['a', 'car', 'was

In [8]:
test_data = WVDataset(test_sents)
test_data.vocab

{'the': 80,
 'cat': 20,
 'sat': 40,
 'on': 40,
 'mat': 20,
 'table': 20,
 'fox': 20,
 'a': 10,
 'car': 10,
 'was': 10,
 'broken': 10}

In [9]:
test_data.ctx_weights

array([0.22500645, 0.07955179, 0.13378963, 0.13378963, 0.07955179,
       0.07955179, 0.07955179, 0.04730178, 0.04730178, 0.04730178,
       0.04730178])

In [10]:
test_data.source2index

{'the': 0,
 'cat': 1,
 'sat': 2,
 'on': 3,
 'mat': 4,
 'table': 5,
 'fox': 6,
 'a': 7,
 'car': 8,
 'was': 9,
 'broken': 10}

### Model

In [44]:
class SGNS():
    def __init__(self, data,dim=100,alpha=0.025, iterations=5):
        self.alpha = alpha
        self.dim = dim
        self.data = data
        self.model = SkipGramNetwork(len(self.data.index2source), len(self.data.index2context), self.dim)
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=alpha)
        self.iterations = iterations
        self.train() 
        self.embs = self.get_embedding()
        
        
    def train(self):
        for epoch in range(self.iterations):    
            epoch_loss = 0
            epoch_pairs = 0
            for (pos_u,pos_v,neg_v) in list(self.data.get_pairs()):
                print(epoch_pairs,end='\r')
                epoch_pairs += 1
                pos_u = torch.LongTensor([pos_u])
                pos_v = torch.LongTensor([pos_v])
                neg_v = torch.LongTensor(neg_v)
                
                self.optimizer.zero_grad()
                loss = self.model.forward(pos_u, pos_v, neg_v)
                epoch_loss += loss
                loss.backward()
                self.optimizer.step()
                
            print("{0:d} epoch of {1:d}, {2:d} pairs, loss: {3:f}".format(epoch+1,self.iterations,epoch_pairs, epoch_loss))
      
    def get_embedding(self):
        e = dict()
        embedding = self.model.u_embeddings.weight.data.numpy()
        for i in range(len(self.data.index2source)):
            e[self.data.index2source[i]] = embedding[i]
        return e
    
    def most_similar(self,to_test,top_n=10):
        emb_test = self.model.u_embeddings(torch.LongTensor([self.data.source2index[to_test]]))
        score = torch.mm(self.model.u_embeddings.weight.data, torch.t(emb_test))
        norms_emb = torch.norm(self.model.u_embeddings.weight, dim=1)
        normalization_factors =  norms_emb * torch.norm(emb_test)
        scores = score.squeeze()/normalization_factors
        values, indices = scores.sort(descending=True)
        values = values.detach().numpy()
        indices = indices.detach().numpy()
        if top_n < 0 or top_n > len(self.data.index2source):
            top_n = len(self.data.index2source)
        for i in range(top_n):
            print(self.data.index2source[indices[i]],':',values[i])

In [45]:
test_data = WVDataset(test_sents,window_size=2,neg_samples=5,sample=0)
model = SGNS(test_data,alpha=0.025,iterations=5,dim=5)
model.most_similar('cat')

1 epoch of 5, 654 pairs, loss: 2667.565918
2 epoch of 5, 640 pairs, loss: 1913.859619
3 epoch of 5, 624 pairs, loss: 1589.992676
4 epoch of 5, 634 pairs, loss: 1508.803833
5 epoch of 5, 637 pairs, loss: 1481.087280
cat : 1.0000001
fox : 0.9931775
on : 0.9506307
mat : 0.861469
table : 0.8440743
a : 0.6433477
sat : 0.63946944
broken : 0.63221645
car : 0.473864
was : 0.44934615


In [46]:
from gensim.models import Word2Vec

m2 = Word2Vec(test_sents, sample=0,compute_loss=True ,min_count=0,vector_size=100,alpha=0.025,min_alpha=0.025,workers=1,sg=1,hs=0,epochs=5,window=2)

In [47]:
m2.wv.most_similar(positive='cat')

[('fox', 0.9956952333450317),
 ('on', 0.9640274047851562),
 ('mat', 0.8453554511070251),
 ('table', 0.8081494569778442),
 ('broken', 0.8074371814727783),
 ('a', 0.7790175676345825),
 ('was', 0.693585991859436),
 ('car', 0.6879597306251526),
 ('sat', 0.5816310048103333),
 ('the', 0.43878644704818726)]

## Apply to 20newsgroups

In [19]:
# shell scripts for downloading the data and placing it in a corresponding directory
!mkdir newsgroups
!curl -o newsgroups/news.tar.gz "http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz"
# extract the files
!gzip -d < newsgroups/news.tar.gz | tar xf - --directory newsgroups

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.7M  100 13.7M    0     0  1908k      0  0:00:07  0:00:07 --:--:-- 2209k00:40 --:--:--  0:00:40  344k


In [34]:
from sklearn.datasets import load_files


twenty_train = load_files('./newsgroups/20news-bydate-train/', encoding='latin1')
twenty_test = load_files('./newsgroups/20news-bydate-test/', encoding='latin1')

In [35]:
twenty_train.data[0]

"From: cubbie@garnet.berkeley.edu (                               )\nSubject: Re: Cubs behind Marlins? How?\nArticle-I.D.: agate.1pt592$f9a\nOrganization: University of California, Berkeley\nLines: 12\nNNTP-Posting-Host: garnet.berkeley.edu\n\n\ngajarsky@pilot.njin.net writes:\n\nmorgan and guzman will have era's 1 run higher than last year, and\n the cubs will be idiots and not pitch harkey as much as hibbard.\n castillo won't be good (i think he's a stud pitcher)\n\n       This season so far, Morgan and Guzman helped to lead the Cubs\n       at top in ERA, even better than THE rotation at Atlanta.\n       Cubs ERA at 0.056 while Braves at 0.059. We know it is early\n       in the season, we Cubs fans have learned how to enjoy the\n       short triumph while it is still there.\n"

In [48]:
step = 10 # use only every step-th document
train_reduced = dict()
test_reduced = dict()
train_reduced['data'] = [twenty_train.data[i] for i in range(0,len(twenty_train.data),step)]
train_reduced['target'] = [twenty_train.target[i] for i in range(0,len(twenty_train.data),step)]
test_reduced['data'] = [twenty_test.data[i] for i in range(0,len(twenty_test.data),step)]
test_reduced['target'] = [twenty_test.target[i] for i in range(0,len(twenty_test.data),step)]

raw_docs = train_reduced['data'] + test_reduced['data']
docs = [list(tokenize(d,lowercase=True)) for d in raw_docs]
len(docs)

1886

In [49]:
%%time

gensim_m = Word2Vec(docs,alpha=0.025, min_alpha=0.025,sg=1,hs=0,epochs=5,vector_size=50)

CPU times: user 10.7 s, sys: 37.1 ms, total: 10.7 s
Wall time: 3.81 s


In [50]:
gensim_m.wv.most_similar(positive='cpu')

[('motherboard', 0.8644053339958191),
 ('mhz', 0.8494293093681335),
 ('isa', 0.8344085216522217),
 ('mounting', 0.8319534659385681),
 ('cache', 0.8281416893005371),
 ('viper', 0.8237494230270386),
 ('maxtor', 0.8235769271850586),
 ('jumper', 0.8224573135375977),
 ('seagate', 0.8213753700256348),
 ('socket', 0.8191305994987488)]

In [51]:
%%time

m = SGNS(WVDataset(docs),dim=50)

1 epoch of 5, 2509273 pairs, loss: 6315909.500000
2 epoch of 5, 2506032 pairs, loss: 5676597.000000
3 epoch of 5, 2507857 pairs, loss: 5518800.500000
4 epoch of 5, 2506791 pairs, loss: 5407366.000000
5 epoch of 5, 2506609 pairs, loss: 5323939.500000
CPU times: user 2h 6min 40s, sys: 26min 2s, total: 2h 32min 43s
Wall time: 2h 28min 33s


In [52]:
m.most_similar('cpu')

cpu : 1.0000001
motherboard : 0.89053804
socket : 0.87638056
seagate : 0.87345856
isa : 0.8704405
ide : 0.8703649
meg : 0.8691706
megs : 0.8687872
floppies : 0.866265
maxtor : 0.8640162


## Remarks
<div class="alert alert-info">
    <ul>
<li>Besides the SkipGram Model, Mikolov introduced a second model, called "Continuous Bag of Words" (CBOW). There, the goal is to predict the current word from the context. The CBOW model is basically the opposite to SkipGram. In SkipGram, we predict the context words (the words in the window), in CBOW, we predict the word in the middle, given the context words (words in the window).</li>
<li>The implementation here is only used for demonstration purposes and not optimized for performance. Hence, it is horribly slow. Also it does not care about memory consumption.</li>
    </ul>
</div>

# Doc2Vec
The SkipGram Model can not only be used to embed words, but also complete sentences, paragraph or documents [1]. The model, corresponding to Skipgram is called PV-DBOW (Paragraph Vector Distributed Bag of Words) in [1]. Given a document, we aim to predict its context, i.e. the words in this document. Documents with similar context (i.e. documents that share many words) should be close in embedding space.  
The second model introduced in [1] is called distributed memory (DM). It works similar to CBOW mentioned above, but we add the document to the context. That means we slide a window over the sentences in the document and try to predict the word in the middle of this window, given the remaining words + the document. As the document information is the same for all word windows, this can be seen as some kind of memory and provided the name distributed memory.

[1] Le, Quoc, and Tomas Mikolov. "Distributed representations of sentences and documents." International conference on machine learning. 2014.

The network architecture stays the same as before, we only change the input layer from words to documents. That is, we do not need to change the computation within our model, but only adapt the data preparation.

In [53]:
from collections import defaultdict
from gensim.models.doc2vec import TaggedDocument
import random

class DVDataset():
    def __init__(self, documents, power=0.75,neg_samples=5,sample=1e-3,min_count=5):
        self.documents = documents
        self.neg_samples = neg_samples
        self.vocab = None
        self.word_vocab = None
        self.index2source = dict() #doc2index
        self.source2index = dict() #index2doc
        self.index2context = dict() #index2word
        self.context2index = dict() #word2index
        self.build_vocab(documents,min_count)
        self.ctx_weights = self.make_freq_table(power)
        self.neg_sample_buffer = []
       
            
    def build_vocab(self,documents,min_count):
        self.vocab = defaultdict(int)
        # word freqency
        raw_vocab = defaultdict(int)
        for d in documents:
            for tag in d.tags:
                self.vocab[tag] += 1
            for word in d.words:
                raw_vocab[word] += 1 
        # only keep words that occur at least min_count times
        self.word_vocab = {k:v for k,v in raw_vocab.items() if v >= min_count}
        del raw_vocab
        
        # map each word to an integer
        for word in self.word_vocab:
            self.context2index[word] = len(self.context2index)
        # reverse mapping (integer to word)
        self.index2context = dict(zip(self.context2index.values(), self.context2index.keys()))
        
        # map each document to an integer
        for doc in self.vocab:
            self.source2index[doc] = len(self.source2index)
        # reverse mapping (integer to document)
        self.index2source = dict(zip(self.source2index.values(), self.source2index.keys()))
            
        
    def make_freq_table(self, power):
        # the unigram distribution raised to 0.75 empirically performed best as negative sampling distribution
        pow_frequency = np.array([self.word_vocab[self.index2context[i]] for i in range(len(self.word_vocab))])**power
        return pow_frequency / pow_frequency.sum()
    
    def negative_sampling(self,context):
        neg = []
        while len(neg) < self.neg_samples:
            try:
                negative = self.neg_sample_buffer.pop()
                # avoid that context word occurs in negative samples
                if negative != context:
                    neg.append(negative)
            except IndexError:
                self.neg_sample_buffer = list(np.random.choice(list(self.index2context.keys()),size=1000,replace=True,p=self.ctx_weights))
        return neg
        
    
    def get_pairs(self):
        for d in self.documents:
            # randomly reject frequent words according to downsampling probability
            words = [self.context2index[w] for w in d.words if w in self.word_vocab]

            for tag in d.tags:
                for w in words:
                    yield self.source2index[tag],w,self.negative_sampling(w)      

### Prepare Data
Create a `TaggedDocument` Object for each documents. This object has two attributes:
- words: a list of the words in the document
- tags: a list of document identifiers (usually a single element list)

In [54]:
from gensim.models.doc2vec import TaggedDocument

doc_id = 0
tagged_docs = []
for d in train_reduced['data'] + test_reduced['data']:
    tagged_docs.append(TaggedDocument(list(tokenize(d,lowercase=True)),[str(doc_id)]))
    doc_id += 1

### Train the model(s)

In [56]:
%%time
d2v = SGNS(DVDataset(tagged_docs,min_count=0),dim=50,iterations=10)

1 epoch of 10, 580512 pairs, loss: 1752520.375000
2 epoch of 10, 580512 pairs, loss: 1428844.125000
3 epoch of 10, 580512 pairs, loss: 1348997.250000
4 epoch of 10, 580512 pairs, loss: 1306878.000000
5 epoch of 10, 580512 pairs, loss: 1269509.125000
6 epoch of 10, 580512 pairs, loss: 1231818.750000
7 epoch of 10, 580512 pairs, loss: 1194996.625000
8 epoch of 10, 580512 pairs, loss: 1158542.375000
9 epoch of 10, 580512 pairs, loss: 1123906.125000
10 epoch of 10, 580512 pairs, loss: 1091808.500000
CPU times: user 1h 11min 54s, sys: 14min 18s, total: 1h 26min 13s
Wall time: 6h 34s


In [57]:
%%time
from gensim.models import Doc2Vec

d2v_gensim = Doc2Vec(tagged_docs,min_count=0,dm=0,vector_size=50,negative=5,epochs=10,hs=0,alpha=0.025,min_alpha=0.025)

CPU times: user 8.1 s, sys: 158 ms, total: 8.26 s
Wall time: 3.87 s


### Evaluate both models on Text Classification

In [58]:
from sklearn.svm import SVC

def eval(model):
    X_train = []
    y_train = []
    X_test = []
    y_test = []

    for i in range(len(train_reduced['target'])):
        X_train.append(model[str(i)])
        y_train.append(train_reduced['target'][i])
    
    for i in range(len(test_reduced['target'])):
        X_test.append(model[str(i+len(train_reduced['target']))])
        y_test.append(test_reduced['target'][i])
        
    clf = SVC(gamma='auto')
    clf.fit(X_train,y_train)
    
    return clf.score(X_test,y_test)    

In [59]:
eval(d2v_gensim)

0.5928381962864722

In [60]:
eval(d2v.get_embedding())

0.5822281167108754

# Deepwalk
The SkipGram model has been adapted to Graphs [1], learning embeddings of nodes in the graph. The key idea here is that nodes that have a similar neighborhood should be close in embeddings space. That corresponds to embeddings of words that occur in a similar context.
In order to achieve this goal, random walks are sampled from the graph and then treated as sentence equivalents.

[1] Perozzi, Bryan, Rami Al-Rfou, and Steven Skiena. "Deepwalk: Online learning of social representations." Proceedings of the 20th ACM SIGKDD international conference on Knowledge discovery and data mining. ACM, 2014.

First, we download a graph dataset.

In [61]:
import os

!mkdir ./cora
!curl -o ./cora/cora.tar.gz "https://linqs-data.soe.ucsc.edu/public/lbc/cora.tgz"
!gzip -d < ./cora/cora.tar.gz | tar xf - --directory ./
!rm ./cora/cora.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  164k  100  164k    0     0  83649      0  0:00:02  0:00:02 --:--:-- 83649


Let's check the description of the dataset.

In [62]:
!cat cora/README

This directory contains the a selection of the Cora dataset (www.research.whizbang.com/data).

The Cora dataset consists of Machine Learning papers. These papers are classified into one of the following seven classes:
		Case_Based
		Genetic_Algorithms
		Neural_Networks
		Probabilistic_Methods
		Reinforcement_Learning
		Rule_Learning
		Theory

The papers were selected in a way such that in the final corpus every paper cites or is cited by atleast one other paper. There are 2708 papers in the whole corpus. 

After stemming and removing stopwords we were left with a vocabulary of size 1433 unique words. All words with document frequency less than 10 were removed.


THE DIRECTORY CONTAINS TWO FILES:

The .content file contains descriptions of the papers in the following format:

		<paper_id> <word_attributes>+ <class_label>

The first entry in each line contains the unique string ID of the paper followed by binary values indicating whether each word in the vocabulary is present (indicated 

### Load the graph with networkx

In [63]:
import networkx as nx

G = nx.read_edgelist('./cora/cora.cites')

### Define a random walk, starting at a particular node

In [64]:
import numpy.random as rand

def random_walk(G,path_length, start=None):
    path = [start]

    while len(path) < path_length:
        cur = path[-1]
        neighbors = list(G.neighbors(cur))
        if len(neighbors) > 0:
            path.append(rand.choice(neighbors))
        else:
            break
    return [str(node) for node in path]

### Create several walks for each node in the graph
Typical values for number of walks per node are 40 and the length of each walk 80

In [65]:
def generate_walks(G,num_paths,path_length):
    walks = []
    for cnt in range(num_paths):
        for node in G.nodes:
            walks.append(random_walk(G,path_length,node))
    return walks

### Feed the generated walks into a Word2VecModel to create embeddings for the nodes

In [66]:
walks = generate_walks(G,4,10)

In [68]:
%%time
dw_gensim = Word2Vec(walks, vector_size=100, min_count=0,epochs=5,sg=1,hs=0)

CPU times: user 2.1 s, sys: 10.9 ms, total: 2.11 s
Wall time: 844 ms


In [69]:
%%time
dw = SGNS(WVDataset(walks,min_count=0),dim=100)

1 epoch of 5, 480698 pairs, loss: 1683031.125000
2 epoch of 5, 482192 pairs, loss: 937517.375000
3 epoch of 5, 481567 pairs, loss: 363318.906250
4 epoch of 5, 481886 pairs, loss: 263049.406250
5 epoch of 5, 481338 pairs, loss: 233549.500000
CPU times: user 24min 48s, sys: 4min 57s, total: 29min 46s
Wall time: 28min 57s


## Node Classification

In [70]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

def eval_dw(model,folds=10,test_size=0.1):
    X = []
    y = []
    with open('./cora/cora.content') as f:
        for line in f.readlines():
            line = line.split()
            y.append(line[-1])
            X.append(model[line[0]])
        clf = LogisticRegression(solver='liblinear',multi_class='ovr')
        scores = []
        for i in range(folds):
            X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=test_size, random_state=i)
            clf.fit(X_train,y_train)
            score = metrics.f1_score(y_test,clf.predict(X_test), average='macro')
            scores.append(score)
        return np.mean(scores)

In [71]:
eval_dw(dw_gensim.wv)

0.8192361404650276

In [72]:
eval_dw(dw.get_embedding())

0.8201304912010012